In [ ]:
!pip install eventregistry
!pip install spacy
!pip install fsspec
!pip install gcsfs 
!pip install text2emotion
!pip install newsapi-python
!pip install cloudstorage
!pip install geocoder

In [ ]:
import requests
import json
import pandas as pd 
from google.colab import auth
from google.cloud import bigquery
from google.cloud import storage
auth.authenticate_user()
import pytz
from datetime import datetime
import geocoder
import time
import pytz
pytz.timezone("Pacific/Auckland")

<DstTzInfo 'Pacific/Auckland' LMT+11:39:00 STD>

In [3]:
projectId = "GCP-Project-ID"
bqclient = bigquery.Client(project=projectId)
storage_client = storage.Client(project=projectId)
bucket = storage_client.get_bucket("together-internals")
# locations_df = pd.read_csv("locations directory")
# motorway_df = pd.read_csv("locations directory")
current = datetime.now(pytz.timezone("Pacific/Auckland"))
current = current.strftime("%Y-%m-%d")


In [1]:
def startDataPull(self):
  def getTrafficData(wendysLocations,viceVersaLocation):
    to_cbd = []
    to_viceversa = [] 
    cbd_urls = []
    viceversa_urls = []
    cbd_names = []
    viceversa_names = []
    address = []
    all_warning_address_cbd = []
    all_warning_address_versa = []
    for x,y in zip(wendysLocations['Highway Coordinates'],wendysLocations['Destinations']):
      url = "https://dev.virtualearth.net/REST/V1/Routes/Driving?o=json&wp.0="+x+"&wp.1="+y+"&avoid=minimizeTolls&key=94PL6GaE9vLv8y9zGgDH~LhrN9nkgmXOpouX0O_FzkQ~AqlA2D3PTDKPE7cK_NuGSE5VMqQb_0h4A5XS_AkfepjQT27zxmqkdNICJp_Knu_a"
      cbd_urls.append(url)
      for x in cbd_urls:
        request = requests.get(x)
        to_cbd.append(request.json())

    for x,y in zip(viceVersaLocation['Highway Coordinates'],viceVersaLocation['Destinations']):
      url = "https://dev.virtualearth.net/REST/V1/Routes/Driving?o=json&wp.0="+x+"&wp.1="+y+"&avoid=minimizeTolls&key=94PL6GaE9vLv8y9zGgDH~LhrN9nkgmXOpouX0O_FzkQ~AqlA2D3PTDKPE7cK_NuGSE5VMqQb_0h4A5XS_AkfepjQT27zxmqkdNICJp_Knu_a"
      viceversa_urls.append(url)
      for x in viceversa_urls:
        request = requests.get(x)
        to_viceversa.append(request.json())


    # TO CBD Data
    to_cbd_df = pd.DataFrame(data=to_cbd)
    #Unnesting
    to_cbd_df = pd.json_normalize(to_cbd_df.explode('resourceSets')['resourceSets'].dropna())
    to_cbd_df = pd.json_normalize(to_cbd_df.explode('resources')['resources'])
    to_cbd_df = pd.json_normalize(to_cbd_df.explode('routeLegs')['routeLegs'])
    to_cbd_df = pd.json_normalize(to_cbd_df.explode('itineraryItems')['itineraryItems'])
    to_cbd_df = to_cbd_df.explode('details')
    cbd_dataList = to_cbd_df['details']
    cbd_dataList = pd.json_normalize(cbd_dataList)
    cbd_dataList = pd.DataFrame(data=cbd_dataList)

    for x in cbd_dataList['names']:
      cbd_names.append(x)

    to_cbd_df['names'] = cbd_names
    to_cbd_df['names'] = to_cbd_df['names'].astype(str)
    to_cbd_df = to_cbd_df[to_cbd_df['names'].str.contains("Motorway", regex=False,na=False)]
    to_cbd_df = to_cbd_df[['warnings','names']].dropna()
    to_cbd_df = to_cbd_df.reset_index()
    all_warning_address_cbd = to_cbd_df['names']
    to_cbd_df = pd.json_normalize(to_cbd_df.explode('warnings')['warnings'])
    to_cbd_df['date'] = current
    to_cbd_df['category'] = "to CBD"
    to_cbd_df = to_cbd_df.drop_duplicates(subset="origin")

    # Vice Versa Data
    to_viceversa_df = pd.DataFrame(data=to_viceversa)
    #Unnesting
    to_viceversa_df = pd.json_normalize(to_viceversa_df.explode('resourceSets')['resourceSets'].dropna())
    to_viceversa_df = pd.json_normalize(to_viceversa_df.explode('resources')['resources'])
    to_viceversa_df = pd.json_normalize(to_viceversa_df.explode('routeLegs')['routeLegs'])
    to_viceversa_df = pd.json_normalize(to_viceversa_df.explode('itineraryItems')['itineraryItems'])
    to_viceversa_df = to_viceversa_df.explode('details')
    viceversa_dataList = to_viceversa_df['details']
    viceversa_dataList = pd.json_normalize(viceversa_dataList)
    viceversa_dataList = pd.DataFrame(data=viceversa_dataList)

    for x in viceversa_dataList['names']:
      viceversa_names.append(x)

    to_viceversa_df['names'] = viceversa_names
    to_viceversa_df['names'] = to_viceversa_df['names'].astype(str)
    to_viceversa_df = to_viceversa_df[to_viceversa_df['names'].str.contains("Motorway", regex=False,na=False)]
    to_viceversa_df = to_viceversa_df[['warnings','names']].dropna()
    to_viceversa_df = to_viceversa_df.reset_index()
    all_warning_address_versa = to_viceversa_df['names']
    to_viceversa_df = pd.json_normalize(to_viceversa_df.explode('warnings')['warnings'])
    to_viceversa_df['date'] = current
    to_viceversa_df['category'] = "Vice Versa"
    to_viceversa_df = to_viceversa_df.drop_duplicates(subset="origin")

    # Push data to BQ
    frames = [to_cbd_df,to_viceversa_df]
    df = pd.concat(frames)
    for x in df['to']:
      g = geocoder.bing(x, method='reverse',key='BING_API_KEY')
      for results in g:
        address.append(g.city)
        time.sleep(0.3)
    all_warning_address = [all_warning_address_cbd,all_warning_address_versa]
    all_warning_address = pd.concat(all_warning_address)
    df['names'] = all_warning_address
    df['city'] = address
    df.to_gbq('baconator.traffic_data',projectId,if_exists='replace')

  getTrafficData(locations_df,motorway_df)

  return 'Traffic Data Pushed!'

  

In [2]:
startDataPull(self=True)